In [2]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scripts.entity_utils import *

In [4]:
dev_pred = [json.loads(line) for line in open('../outputs/test_folder/combined.jsonl')]
dev_data = [json.loads(line) for line in open('../model_data/pwc_split_on_sectioned/dev.jsonl')]

dev_data = {x['doc_id']:x for x in dev_data}
for x in dev_pred :
    x['true_coref'] = dev_data[x['doc_id']]['coref']
    x['n_ary_relations'] = [Relation(*y)._asdict() for y in dev_data[x['doc_id']]['n_ary_relations']]
    x['true_entities'] = {}
    for entity in used_entities :
        x['true_entities'][entity] = list(set([rel[entity] for rel in x['n_ary_relations']]))
        
    x['gold_to_type'] = {}
    for rel in x['n_ary_relations'] :
        for k, v in rel.items() :
            x['gold_to_type'][v] = k

In [5]:
for v in dev_pred :
    v['prediction'] = {tuple(x['span']):x['label'] for x in v['prediction']}
    v['gold'] = {tuple(x['span']):x['label'] for x in v['gold']}

In [6]:
list(enumerate([x['doc_id'] for x in dev_pred]))

[(0, '0116899fce00ffa4afee08b505300bb3968faf9f'),
 (1, '02e85d62fbd8249a046d00ac10e39546511b2a51'),
 (2, '04640006606ddfb9d6aa4ce8f55855b1f23ec7ed'),
 (3, '0523e14247d74c4505cd5e32e1f0495f291ec432'),
 (4, '060ff1aad5619a7d6d6cdfaf8be5da29bff3808c'),
 (5, '0678a8abea82793993cd89383319da75f6dc4be3'),
 (6, '07f3f736d90125cb2b04e7408782af411c67dd5a'),
 (7, '088b649415378c4dde666f614ba24d08a36b6652'),
 (8, '09879f7956dddc2a9328f5c1472feeb8402bcbcf'),
 (9, '0a3381f0432c5cfe491c718349d7a44e5814592c'),
 (10, '0a34fe39e9938ae8c813a81ae6d2d3a325600e5c'),
 (11, '0b0cf7e00e7532e38238a9164f0a8db2574be2ea'),
 (12, '0b4b6932d5df74b366d9235b40334bc40d719c72'),
 (13, '0c5c93594647d77dcbad5f3ed71139b186a23a3c'),
 (14, '0c9ae806059196007938f24d0327a4237ed6adf5'),
 (15, '0cfdcf2a0e345cdf7e680c30d136fdedb0eccb28')]

In [7]:
ex = dev_pred[11]

In [8]:
ex['doc_id']

'0b0cf7e00e7532e38238a9164f0a8db2574be2ea'

In [16]:
from dygie.models.global_analysis.clustering import *
from dygie.models.global_analysis.relation_extraction import *

In [19]:
from dygie.training.evaluation import *
cluster_matching_thresholds = generate_thresholds(dev_data.values())
cluster_matching_thresholds

{'Material': 1.0, 'Metric': 1.0, 'Task': 1.0, 'Method': 1.0}

In [11]:
for ex in dev_pred :
    clusters, spl, cluster_labels, linked_clusters = do_clustering(ex, 'prediction', 'coref_prediction', plot=False)
    ex['clusters'] = clusters
    relation_matrix = get_relations_between_clusters(ex, 'prediction', cluster_labels, ex['relation_threshold'])
    ex['predicted_relations'] = {n:generate_relations(clusters, relation_matrix, linked_clusters, n)
                                 for n in range(1, 6)}
    
    ex['gold_clusters'] = {k:{'spans' : v, 'words' : [" ".join(ex['words'][y[0]:y[1]]) for y in v]} for k, v in ex['true_coref'].items()}
    ex['true_relations'] = {n:generate_true_relations(ex['n_ary_relations'], n)
                                 for n in range(1, 6)}
    
    map_all_clusters_to_true(ex, cluster_matching_thresholds)

100%|██████████| 90/90 [00:00<00:00, 773.56it/s]


In [ ]:
from scripts.convert_predictions_to_brat import *
generate_brat_annotations(documents={x['doc_id']:x for x in dev_pred}, 
                          brat_anno_folder='../brat/data/result_extraction_results/ner')

In [12]:
evaluate_for_all_document(dev_pred)

100%|██████████| 90/90 [00:00<00:00, 766.07it/s]


(0.38527023189695814, 0.44475819742674744, 0.4076443775746348)

In [13]:
import pandas as pd

In [42]:
gold_res, pred_res = run_eval_for_relation_extraction(dev_pred)

In [43]:
for n, v in gold_res.items() :
    if n < 5 :
        print(n)
        print(pd.DataFrame(v))
        print('-'*20)
        print(pd.DataFrame(pred_res[n]))
        print("="*20)

1
    Material    Metric      Task    Method
f1  0.481481  0.409091  0.500000  0.585366
p   0.590909  0.500000  0.526316  0.666667
r   0.406250  0.346154  0.476190  0.521739
--------------------
    Material    Metric      Task    Method
f1  0.419355  0.391304  0.318182  0.340426
p   0.433333  0.450000  0.304348  0.333333
r   0.406250  0.346154  0.333333  0.347826
2
    Material,Metric  Material,Task  Material,Method  Metric,Task  \
f1         0.205882       0.266667         0.317460     0.250000   
p          0.304348       0.347826         0.434783     0.300000   
r          0.155556       0.216216         0.250000     0.214286   

    Metric,Method  Task,Method  
f1       0.290909     0.326531  
p        0.400000     0.380952  
r        0.228571     0.285714  
--------------------
    Material,Metric  Material,Task  Material,Method  Metric,Task  \
f1         0.176471       0.153846         0.194444     0.166667   
p          0.260870       0.178571         0.218750     0.375000   
r

In [96]:
for s1, s2, p in dev_pred[11]['coref_prediction'] :
    w1 = " ".join(dev_pred[11]['words'][s1[0]:s1[1]])
    w2 = " ".join(dev_pred[11]['words'][s2[0]:s2[1]])
    if 'WMT' in w1 and 'WMT' in w2 :
        print(w1, w2, p)

WMT 2014 English - French dataset WMT 2014 English - to - German 0.998
WMT 2014 English - German dataset WMT 2014 English - French dataset 1.0
WMT 2014 English - German dataset WMT 2014 English - to - German 1.0
WMT 2014 English - to - French translation task WMT 2014 English - to - French translation task 0.999
WMT 2014 English - to - French translation task WMT 2014 English - to - French translation tasks 0.999
WMT 2014 English - to - German translation task WMT 2014 English - to - French translation tasks 0.997
WMT 2014 English - to - German translation task WMT 2014 English - to - German translation task 1.0
WMT 2014 English - to - French translation task WMT 2014 English - to - German translation task 0.998
WMT 2014 English - to - German translation task WMT 2014 English - to - French translation task 0.998
WMT 2014 English - to - French translation task WMT 2014 English - to - French translation tasks 0.999
WMT 2014 English - to - German translation task WMT 2014 English - to - F